<a href="https://colab.research.google.com/github/TomoharuKurosu/Tomoharu_DS2/blob/main/%E6%AF%8D%E9%9B%86%E5%9B%A3%E4%BD%9C%E6%88%90%EF%BC%BF%E3%83%A1%E3%83%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %% init
from setting import *
import unicodedata #このライブラリを使用すると、文字の名前を取得したり、Unicodeカテゴリを調べたり、文字を正規化（Normalization）したりすることができ
# %% ufd
def key_clnser(df): #特定のカラム名を一括で宛名番号に変更します
    return df.rename(columns={
        "住民コード":"宛名番号",
        "住民番号":"宛名番号",
        "個人番号":"宛名番号",
    })
def converter(s): #入力された文字列(s)をUnicode正規化形式NFKC（Compatibility Composition）に基づいて正規化します。
    return unicodedata.normalize("NFKC", s)
# %%
p_in = "../data_raw/市民室_1ファイル/001_住基データ.csv"
p_out = "../data_clns+/0.母集団.csv"

def gakunen(birthday):
    '''
    生年の補正
    '''
    birthday = str(birthday) #birthday = str(birthday)で、生年月日を文字列として扱います。これにより、数値でも文字列でも引数として受け取れるようにしてい
    y = int(birthday[:4])#年（最初の4文字）を整数に変換して取得します。
    m = int(birthday[4:6])#月（次の2文字）を整数に変換して取得します。
    d = int(birthday[6:])#日（残りの文字）を整数に変換して取得します。ｄｄ
    if m < 4:
        return y - 1
    if m == 4 and d == 1:#4月1日生まれも特例として前年の学年に含める
        return y - 1
    return y

_df = (
    # 入力
    pd
    .read_csv(p_in)
    .pipe(key_clnser)#key_clnser関数を適用している
    [["宛名番号", "世帯番号", "生年月日"]]#指定した列（「宛名番号」「世帯番号」「生年月日」）だけを抽出する操作
    .loc[lambda df: df["生年月日"] >= 19900101]#、「生年月日」が1990年1月1日以上の行のみを選択しています。つまり、1990年1月1日以降に生まれた人々のデータを抽出しています。
    .sort_values("生年月日")#データフレームを「生年月日」に基づいて昇順にソートします。
    .drop_duplicates()#データフレーム内で重複する行を削除します
    # 加工   assignメソッドを使って、新しい列「生年」をデータフレームに追加しています。
    .assign(生年 = lambda df: df["生年月日"].astype(str).apply(gakunen))
    .merge(
        pd.DataFrame({"年度":[i for i in range(2011, 2024)]})
        , how="cross"
    )#れにより、各「宛名番号」と「年度」のすべての組み合わせが作成されます。
    .assign(学年齢 = lambda df: df["年度"] - df["生年"])
    .loc[lambda df: df["学年齢"].between(0, 15)]#locメソッドで、「学年齢」が0から15の間にある行のみをフィルタリングしています。
    [["宛名番号", "世帯番号", "年度", "学年齢"]]
    .sort_values(["宛名番号", "年度"])#各「宛名番号」について年度順に並べ替えられ、データがより見やすく整理されます。
    .pipe(check_df)
)
df_pop = _df#rename
_df.to_csv(p_out, index=False)#データフレーム _df を、指定されたパス p_out にCSV形式で保存しています。index=False は、データフレームの行番号（インデックス）をファイルに含めないようにするオプションです。
pd.read_csv(p_out)

# %%
p_in = "../data_raw/要対協データ_1ファイル/要対協.csv"
df_yotaikyo = (
    # 入力
    pd
    .read_csv(p_in)
    .pipe(key_clnser)
    [["宛名番号", "年度", "種別"]]
    # 加工
    .assign(年度 = lambda df: pd.to_datetime(df["年度"]).dt.year)
    .loc[lambda df: df["種別"].notna()]#locメソッドを使用して、「種別」列がNaN（欠損値）ではない行だけをフィルタリングしています。
    .assign(種別 = lambda df:
        df["種別"]
        .apply(converter)
        .str.replace("、", "")
        .str.replace(",", "")
        .str.replace("疑い", "")
        .str.replace("疑", "")
    #「、」（日本語の読点）を削除、「,」（カンマ）を削除、「疑い」という文字を削除、「疑」という文字を削除
    .assign(要対協_被身体的虐待数 = lambda df: df["種別"].str.contains("B") * 1)#「種別」列の値に「B」が含まれているかどうかを確認し、含まれていれば1、含まれていなければ0を格納します。
    .assign(要対協_被心的虐待数 = lambda df: df["種別"].str.contains("E") * 1)
    .assign(要対協_被性的虐待数 = lambda df: df["種別"].str.contains("S") * 1)
    .assign(要対協_被ネグレクト数 = lambda df: df["種別"].str.contains("N") * 1)
    .drop(columns="種別")#ここで「種別」列を削除します
    .groupby(["宛名番号", "年度"], as_index=False)#「宛名番号」と「年度」でグループ化し、各グループに対して数値列（各虐待数）の合計を計算します。
    .sum()
    .assign(要対協_被虐待数 = lambda df: df.filter(regex="要対協_").sum(axis=1))#、各行の合計虐待数が計算されます。２つの虐待を受けている可能性もある
    .assign(要対協_被虐待フラグ = lambda df: (df["要対協_被虐待数"] > 0) * 1)#「要対協_被虐待数」が1以上の場合、1（虐待あり）を、0の場合は0（虐待なし）を格納します。
    # 出力
    .loc[lambda df: df["要対協_被虐待フラグ"] > 0]#locメソッドを使用して、「要対協_被虐待フラグ」列の値が0より大きい（つまり、虐待があった）行だけを抽出
    .drop_duplicates()#重複する行を削除します。
    .pipe(check_df)
)

# %%
p_out = "../data_clns+/1.要対協_目的変数.csv"

_df = (
    df_yotaikyo
    [["宛名番号", "年度", "要対協_被虐待フラグ"]]
    .merge(
        df_pop
        [["宛名番号", "年度"]]
        , on=["宛名番号", "年度"]
        , how="right"
    )
    .fillna(0)
    .pipe(check_df)
)

_df.to_csv(p_out, index=False)
pd.read_csv(p_out)
# %%
p_out = "../data_clns+/1.要対協_昨年度累計.csv"
def cumsum(df):
    df = df.sort_values(["宛名番号", "年度"])
    return (
        df
        [["宛名番号", "年度"]]
        .join(
            df
            .drop(columns="年度")
            .groupby("宛名番号")
            .cumsum()
            .rename(columns=lambda c: c+"_昨年度累計")
        )
    )
def ffill(df):
    df = df.sort_values(["宛名番号", "年度"])
    return (
        df
        [["宛名番号", "年度"]]
        .join(
            df
            .drop(columns="年度")
            .groupby("宛名番号")
            .fillna(method="ffill")
            .fillna(0)
        )
    )
_df = (
    df_yotaikyo
    .drop(columns="要対協_被虐待フラグ")
    .sort_values(["宛名番号", "年度"])
    .assign(年度 = lambda df: df["年度"] + 1)
    .pipe(cumsum)
    .merge(
        df_pop
        [["宛名番号", "年度"]]
        , on=["宛名番号", "年度"]
        , how="outer"
    )
    .pipe(ffill)
    .pipe(check_df)
)

_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
_df_yotaikyo = (
    # 入力
    pd
    .read_csv(p_in)
    .pipe(key_clnser)
    [["宛名番号", "年度", "種別"]]
    # 加工
    .assign(年度 = lambda df: pd.to_datetime(df["年度"]).dt.year)
    .loc[lambda df: df["種別"].notna()]
    .assign(種別 = lambda df:
        df["種別"]
        .apply(converter)
        .str.replace("、", "")
        .str.replace(",", "")
        .str.replace("疑い", "")
        .str.replace("疑", "")
    )
    .assign(要対協_A数 = lambda df: (df["種別"] == "A") * 1)
    .assign(要対協_ハイリスク数 = lambda df: df["種別"].str.contains("ハイリスク") * 1)
    .assign(要対協_特定妊婦 = lambda df: df["種別"].str.contains("特定妊婦") * 1)
    .drop(columns="種別")
    .groupby(["宛名番号", "年度"], as_index=False)
    .sum()
    # # 出力
    .loc[lambda df: df.filter(regex="要対協_").sum(axis=1) > 0]
    # ["種別"].unique()
    .drop_duplicates()
    .pipe(check_df)
)
# %%
p_out = "../data_clns+/1.要対協_昨年度累計_その他.csv"
def cumsum(df):
    df = df.sort_values(["宛名番号", "年度"])
    return (
        df
        [["宛名番号", "年度"]]
        .join(
            df
            .drop(columns="年度")
            .groupby("宛名番号")
            .cumsum()
            .rename(columns=lambda c: c+"_昨年度累計")
        )
    )
def ffill(df):
    df = df.sort_values(["宛名番号", "年度"])
    return (
        df
        [["宛名番号", "年度"]]
        .join(
            df
            .drop(columns="年度")
            .groupby("宛名番号")
            .fillna(method="ffill")
            .fillna(0)
        )
    )
_df = (
    _df_yotaikyo
    # .drop(columns="要対協_被虐待フラグ")
    .sort_values(["宛名番号", "年度"])
    .assign(年度 = lambda df: df["年度"] + 1)
    .pipe(cumsum)
    .merge(
        df_pop
        [["宛名番号", "年度"]]
        , on=["宛名番号", "年度"]
        , how="outer"
    )
    .pipe(ffill)
    .pipe(check_df)
)

_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
(
    pd
    .read_csv(p_out)
    .iloc[:, -3:]
    .describe()
)